# Interactive Visual Analytics Dashboard with Ploty Dash

In this lab, you will be building a Plotly Dash application for users to perform interactive visual analytics on SpaceX launch data in real-time.

This dashboard application contains input components such as a dropdown list and a range slider to interact with a pie chart and a scatter point chart.

In [3]:
!pip install dash==1.19.0  

!pip install jupyter_dash 

!pip install --upgrade plotly

  Using cached dash-1.19.0-py3-none-any.whl
  Using cached dash_table-4.11.2-py3-none-any.whl
  Using cached dash_core_components-1.15.0-py3-none-any.whl
  Using cached Flask_Compress-1.10.1-py3-none-any.whl (7.9 kB)
  Using cached dash_html_components-1.1.2-py3-none-any.whl
  Using cached dash_renderer-1.9.0-py3-none-any.whl
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for dash: filename=dash-1.19.0-py3-none-any.whl size=84021 sha256=3ea810ed9f528e1219a778d8e0ea26266733e6a9b8f7e478a08b32119cbfa4a2
  Stored in dir

In [1]:
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash
import plotly.graph_objects as go
import plotly.express as px


In [2]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [3]:
# Create a dash application
app = JupyterDash(__name__)

In [4]:
# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard, by Dan Barstow B.S. Mathematics: Operations Research',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(
                                        id='site-dropdown',
                                        options=[
                                            {'label': 'All Sites', 'value': 'ALL'},
                                            {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                            {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                            {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                            {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                        ],
                                        value='ALL',
                                        placeholder="Select a Launch Site",
                                        searchable=True
                                    ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=1000,
                                                marks={0: '0',
                                                       2500: '2500',
                                                       5000: '5000',
                                                       7500: '7500'},
                                                value=[min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie(value):
    if value == 'ALL':
        df = spacex_df
        fig = px.pie(df, values='class',
        names='Launch Site',
        title='Total Success Launches By Site')
        return fig
    else:
        df = spacex_df[spacex_df['Launch Site'] == value]
        filtered_df = df.groupby('class').size().reset_index(name='class count')
        fig = px.pie(filtered_df, values='class count',
        names='class',
        title='Total Success Launches By %s' % value)
        return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'),
               Input(component_id="payload-slider", component_property="value")])
def get_scatter(value, slider_range):
    low, high = slider_range
    if value == 'ALL':
        df = spacex_df
        mask = (df['Payload Mass (kg)'] > low) & (df['Payload Mass (kg)'] < high)
        fig = px.scatter(df[mask], x="Payload Mass (kg)", y="class", 
                         color="Booster Version Category",
                         title='Correlation between Payload and Success for all Sites')
        return fig
    else:
        df = spacex_df[spacex_df['Launch Site'] == value]
        mask = (df['Payload Mass (kg)'] > low) & (df['Payload Mass (kg)'] < high)
        fig = px.scatter(df[mask], x="Payload Mass (kg)", y="class", 
                         color="Booster Version Category",
                         title='Correlation between Payload and Success for %s' % value)
        return fig   
    

# Run the app
if __name__ == '__main__':
    app.run_server(mode='jupyterlab', 
                   port = 8090,
                   dev_tools_ui=True, 
                   debug=True, 
                   dev_tools_hot_reload =True, 
                   threaded=True)

_Completed by Dan Barstow, B.S. Mathematics: Operations Research_